In [ ]:

from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
import os
import json

os.chdir(project_root)
fig_dir = os.path.join(project_root, 'results/paper_figures/')
import sys
sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)
sys.path.insert(0,project_root)

from utils.fig_3_functions import make_mouse_df_directionality, plot_directionalities, make_mouse_df,make_mouse_df_directionality_nonvisual
from utils.fig_3_functions import make_monkey_df_directionality,plot_directionalities_densities,plot_activitytype_by_direction_densities,residualize_within_condition_multi
from utils.stats_functions import get_comparison_test_stars

save_figs = False

## upload

In [ ]:
mouse_stats_path = os.path.join(project_root, 'results/fig_3',f'mouse_stats.pkl')
with open(mouse_stats_path, 'rb') as f:
	mouse_stats = pickle.load(f)


monkey_names = ['L','A','D']
all_monkey_stats = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_3',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats[monkey_name] = pickle.load(f)

## Mouse Directionality

### plotting

In [ ]:
dir_mouse_df_stimulus = make_mouse_df_directionality(mouse_stats_=mouse_stats)


In [ ]:
#Fig 3A
plot_directionalities(dir_mouse_df_stimulus[dir_mouse_df_stimulus['Mouse Name']=='MP033'].reset_index(drop=True), x='Area', neuron_property='Split-half r', 
                    neuron_property_label='split-half r', linewidth=0, plot_control_line=False, fig_size=(0.6,1.3),
                    plot_type='stripplot', size=2, alpha=0.3)
plt.yticks([0.8,0.9,1])
if save_figs is True:
    plt.savefig(fig_dir +'fig3_mouse_control_relis.pdf',transparent=True,bbox_inches='tight' )


In [ ]:
#Fig 3B
plot_directionalities(dir_mouse_df_stimulus, x='Direction', neuron_property='EV', 
                    neuron_property_label='EV fraction', linewidth=0, mouse_or_date='Mouse Name', print_pval=True, verbose=True)
if save_figs is True:
    plt.savefig(fig_dir +'fig3_mouse_directionality_evars_inter.pdf',transparent=True,bbox_inches='tight')


#### plot supplemental material

In [ ]:
mouse_df_all = make_mouse_df(mouse_stats)
mouse_df=mouse_df_all[(mouse_df_all['SNR']>2) & (mouse_df_all['Split-half r']>0.8)]

In [ ]:
#Fig Supp 2A
plot_directionalities(mouse_df, x='Area', neuron_property='Split-half r', fig_size=(0.6,1.3),
                      neuron_property_label='split-half r', plot_control_line=False, fill=False, linewidth=0.5, 
                      log_scale=False, impose_y_lim=True)
if save_figs is True:
    plt.savefig(fig_dir +'fig3_supp_mouse_comparisons_trad_reli.pdf',transparent=True,bbox_inches='tight')


In [ ]:
#Fig Supp 2B
plot_directionalities(mouse_df, x='Area', neuron_property='SNR', fig_size=(0.6,1.3),
                      neuron_property_label='SNR', plot_control_line=False, fill=False, linewidth=0.5, 
                      log_scale=True, impose_y_lim=False, height=0.93)
if save_figs is True:
    plt.savefig(fig_dir +'fig3_supp_mouse_comparisons_SNR.pdf',transparent=True,bbox_inches='tight')


In [ ]:
#Fig Supp 2C
plot_directionalities(mouse_df, x='Area', neuron_property='1-vs-rest r²', fig_size=(0.6,1.3),
                      neuron_property_label='1-vs-rest r²', plot_control_line=False, fill=False, linewidth=0.5, 
                      log_scale=False, impose_y_lim=False, height=0.95)

if save_figs is True:
    plt.savefig(fig_dir +'fig3_supp_mouse_comparisons_one_vs_rest.pdf',transparent=True,bbox_inches='tight')


In [ ]:
#Fig Supp 2D
plot_directionalities(mouse_df, x='Area', neuron_property='Variance across stimuli', fig_size=(0.6,1.3),
                      neuron_property_label=r'$\text{var}_{\text{stim}}$', plot_control_line=False, fill=False, linewidth=0.5, 
                      log_scale=True, impose_y_lim=False, height=0.95)

if save_figs is True:
    plt.savefig(fig_dir +'fig3_supp_mouse_comparisons_var_stim.pdf',transparent=True,bbox_inches='tight')


In [ ]:
#Fig Supp 2E
plot_directionalities(mouse_df, x='Area', neuron_property='Variance w/in stimulus\nacross trials', fig_size=(0.6,1.3),
                      neuron_property_label=r'$\text{var}_{\text{repeats}}$', plot_control_line=False, fill=False, linewidth=0.5, 
                      log_scale=True, impose_y_lim=False, height=0.95)

if save_figs is True:
    plt.savefig(fig_dir +'fig3_supp_mouse_comparisons_var_repeats.pdf',transparent=True,bbox_inches='tight')


### gray screen 

In [ ]:
nonvisual_neurons = True
dir_mouse_df_stimulus_non_visual = make_mouse_df_directionality_nonvisual(mouse_stats_=mouse_stats, nonvisual_neurons=nonvisual_neurons, dataset_types=['ori32_spont','natimg32_spont'])
dir_mouse_df_stimulus_visual = make_mouse_df_directionality_nonvisual(mouse_stats_=mouse_stats, nonvisual_neurons=False, dataset_types=['ori32_spont','natimg32_spont'])
dir_mouse_df_stimulus_non_visual['Neuron Type'] = 'Non-visual'
dir_mouse_df_stimulus_visual['Neuron Type'] = 'Visual'
dir_mouse_df_stimulus = pd.concat([dir_mouse_df_stimulus_non_visual, dir_mouse_df_stimulus_visual], ignore_index=True)


In [ ]:
#Fig Supp 3A
fig_size=(2,1.3)
fig, axes = plt.subplots(1,2, figsize=fig_size, sharey=True)
plot_directionalities_densities(dir_mouse_df_stimulus_visual, ax=axes[0],x='Direction', neuron_property='EV', 
                    neuron_property_label='EV fraction', linewidth=0, 
                    height=0.9, y_offset=-0.1)
axes[0].set_title('Visual\nneurons', fontsize=6*1.1 , y=0.95)
plot_directionalities_densities(dir_mouse_df_stimulus_non_visual, ax=axes[1], x='Direction', neuron_property='EV', 
                    neuron_property_label='EV fraction', linewidth=0, 
                    height=0.9, y_offset=-0.1, mouse_or_date='Mouse Name')
plt.subplots_adjust(wspace=0.1)
axes[1].set_title('Non-visual\nneurons', fontsize=6*1.1, y=0.95)
if save_figs is True:
    plt.savefig(fig_dir +'fig3_mouse_directionality_evars_inter_nonvisual.pdf',transparent=True,bbox_inches='tight')


### residuals

In [ ]:
fontsize=6
hue='Direction'
x ='Activity Type'
neuron_property = 'EV_resid'

mouse_directionality_df = make_mouse_df_directionality(mouse_stats, dataset_types=['ori32','natimg32','ori32_spont','natimg32_spont'])
# 1) Control for many numeric covariates
covs = ['SNR', 'Split-half r',
       'Variance across stimuli',
       'Variance w/in stimulus\nacross trials']

dfr = pd.DataFrame()
for mouse in mouse_directionality_df['Mouse Name'].unique():
	mini = mouse_directionality_df[mouse_directionality_df['Mouse Name']==mouse]
	dfr_ = residualize_within_condition_multi(
    mini, y='EV', xs=covs, cond='Activity Type', flag_col='control_shuffle', flag_keep=False
	)
	dfr = pd.concat((dfr,dfr_))

In [ ]:
# Fig Supp 3E
hue='Direction'
hue_order = dfr[hue].unique()
fig, ax = plt.subplots(figsize=(1.8,1.3))
sns.violinplot(data=dfr, x=x,hue=hue, palette=("#72BEB7", "#EDAEAE"), cut=0,
               order=dfr['Activity Type'].unique(), hue_order=hue_order, 
               y=neuron_property,saturation=1, ax=ax, inner_kws={'box_width':2, 'whis_width':0.5,
									'marker':'_', 'markersize':3,
									'markeredgewidth':0.8,
									},linewidth=0)
ax.legend_.remove()
sns.despine()
# cosmetics
sns.despine()
ax.set_xlabel(None)
ax.set_ylabel('EV residual', fontsize=fontsize, labelpad=-1.75)
ax.tick_params(axis='x', labelsize=fontsize, width=0.5, length=2, pad=1)
ax.tick_params(axis='y', labelsize=fontsize, width=0.5, length=2, pad=1)
ax.set_xticklabels(['stimulus','gray\nscreen'], fontsize=6)

for spine in ax.spines.values():
	spine.set_linewidth(0.3)

order = dfr['Activity Type'].unique()
# compute y for stars a bit above the taller of the pair
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in order:
	dfr_median = dfr[dfr[x]==x_i].groupby(['Mouse Name','Dataset Type','Area','Direction'])[neuron_property].median().reset_index()
	pvals[x_i] = get_comparison_test_stars(dfr_median, independent_variable=hue, neuron_property=neuron_property,
												mouse_or_date='Mouse Name',perm_type='paired')
for i, act in enumerate(order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = dfr[dfr[x]==act][neuron_property].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
	if act =='Variance across stimuli':
		y_top = y_top + 0.08*yrange
		# print('adjusting y_top for variance across stimuli')
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=6, color=color)
# ax.set_ylim(bottom=-0.23, top=0.27)
if save_figs is True:
	plt.savefig(fig_dir + f'fig3_supp_mouse_comparisons_evar_residuals_stim_GSLO.pdf',transparent=True,bbox_inches='tight' )

## Macaque L Directionality

### plotting

In [ ]:
monkey_name = 'L'
monkey_stats = all_monkey_stats[monkey_name]
dir_monkey_df_stimulus = make_monkey_df_directionality(monkey_stats)

In [ ]:
#Fig 3C
plot_directionalities(dir_monkey_df_stimulus[(dir_monkey_df_stimulus['Dataset Type']=='SNR')].reset_index(drop=True), x='Area', neuron_property='Split-half r', 
                    neuron_property_label='split-half r', linewidth=0, plot_control_line=False, fig_size=(0.6,1.3),
                    plot_type='stripplot', size=2, alpha=0.3, animal='monkey')
plt.yticks([0.8,0.9,1])
if save_figs is True:
    plt.savefig(fig_dir +'fig3_monkey_control_relis.pdf',transparent=True,bbox_inches='tight' )


In [ ]:
#Fig 3D
plot_directionalities_densities(dir_monkey_df_stimulus, x='Direction', neuron_property='EV', 
                    neuron_property_label='EV fraction', linewidth=0, animal='monkey', height=0.95,
                    perm_type='paired')
if save_figs is True:
    plt.savefig(fig_dir +'fig3_monkey_control_evars.pdf',transparent=True,bbox_inches='tight' )


#### supplemental

In [16]:
from utils.fig_6_functions import make_monkey_df_neuron_properties
resp_dataset_type_dict = {'A': ['SNR','RF_thin', 'RF_large'],
                    'L': ['SNR','RF_thin', 'RF_large'],
                    'D': ['SNR']}

monkey_names = ['L','A','D']
all_monkey_stats_fig6 = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_6',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats_fig6[monkey_name] = pickle.load(f)
        
all_monkey_df_properties = pd.DataFrame()
for monkey_name in ['L','A','D']:
	monkey_stats = all_monkey_stats_fig6[monkey_name]
	df_monkey_ = make_monkey_df_neuron_properties(monkey_stats, dataset_types=resp_dataset_type_dict[monkey_name])
	df_monkey_['Monkey Name'] = monkey_name
	all_monkey_df_properties = pd.concat([all_monkey_df_properties, df_monkey_], ignore_index=True)



In [ ]:
# Fig Supp 2F
filt_monkeys_df_properties = all_monkey_df_properties[all_monkey_df_properties['control_shuffle']==False].copy()
x_order = ['L','A','D']
xticklabels = {'L':'L','A':'A','D':'D'}
hue_order = ['V4','V1']
hue = 'Area'	
y = 'split-half r'
neuron_property_label = y
x='Monkey Name'
fontsize=6
impose_y_lim = True
palette = ("#72BEB7", "#EDAEAE")
fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.violinplot(data=filt_monkeys_df_properties, x=x, y=y, hue=hue, ax=ax, alpha=0.7,
               inner_kws={'box_width':1.5, 'whis_width':0.7,
                                    'marker':'_', 'markersize':3,
                                    'markeredgewidth':0.5,
                                    },linewidth=0.5,cut=0,
               palette=palette, saturation=1, fill=False)
sns.despine()
ax.tick_params(axis='both', labelsize=fontsize, width=0.5, length=2, pad=1.5)


ax.spines[:].set_linewidth(0.3)

ax.set(xlabel=None)
ax.set_ylabel(neuron_property_label, fontsize=fontsize, labelpad=-4)
ax.set_xticks(ticks=ax.get_xticks(), labels=[xticklabels.get(lbl, lbl) for lbl in x_order])
if impose_y_lim is True:
	# Get the y-axis ticks
	y_ticks = plt.gca().get_yticks()
	# Check if 1 is among the ticks
	if 1 in y_ticks:
		ax.set_ylim(top=1)

ax.legend(title=None, fontsize=fontsize*0.8, labelspacing=0.2, handletextpad=0.3, borderpad=0.3)
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in x_order:
	pvals[x_i] = get_comparison_test_stars(filt_monkeys_df_properties[filt_monkeys_df_properties[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
											mouse_or_date='Date', perm_type='ind')
for i, act in enumerate(x_order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = filt_monkeys_df_properties[filt_monkeys_df_properties[x]==act][y].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
	if stars =="n.s.":
		y_top += 0.02*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)
	ax.legend_.get_frame().set_linewidth(0.3)
if save_figs:
	plt.savefig(os.path.join(fig_dir, 'figS3_monkey_split_half_by_area_all.pdf'), bbox_inches='tight', dpi=300, transparent=True)

In [ ]:
# Fig Supp 2G
filt_monkeys_df_properties = all_monkey_df_properties[all_monkey_df_properties['control_shuffle']==False].copy()
x_order = ['L','A','D']
xticklabels = {'L':'L','A':'A','D':'D'}
hue_order = ['V4','V1']
hue = 'Area'	
y = 'SNR'
neuron_property_label = y
x='Monkey Name'
fontsize=6
impose_y_lim = False
palette = ("#72BEB7", "#EDAEAE")
fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.violinplot(data=filt_monkeys_df_properties, x=x, y=y, hue=hue, ax=ax, alpha=0.7,
               inner_kws={'box_width':1.5, 'whis_width':0.7,
                                    'marker':'_', 'markersize':3,
                                    'markeredgewidth':0.5,
                                    },linewidth=0.5,cut=0,
               palette=palette, saturation=1, fill=False, log_scale=True)
sns.despine()
ax.tick_params(axis='both', labelsize=fontsize, width=0.5, length=2, pad=1.5)


ax.spines[:].set_linewidth(0.3)

ax.set(xlabel=None)
ax.set_ylabel(neuron_property_label, fontsize=fontsize, labelpad=-1)
ax.set_xticks(ticks=ax.get_xticks(), labels=[xticklabels.get(lbl, lbl) for lbl in x_order])
if impose_y_lim is True:
	# Get the y-axis ticks
	y_ticks = plt.gca().get_yticks()
	# Check if 1 is among the ticks
	if 1 in y_ticks:
		ax.set_ylim(top=1)

ax.legend(title=None, fontsize=fontsize*0.8, labelspacing=0.2, handletextpad=0.3, borderpad=0.3)
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in x_order:
	pvals[x_i] = get_comparison_test_stars(filt_monkeys_df_properties[filt_monkeys_df_properties[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
											mouse_or_date='Date', perm_type='ind')
for i, act in enumerate(x_order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = filt_monkeys_df_properties[filt_monkeys_df_properties[x]==act][y].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
	if stars =="n.s.":
		y_top += 0.02*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)
	ax.legend_.get_frame().set_linewidth(0.3)
if save_figs:
	plt.savefig(os.path.join(fig_dir, 'figS3_monkey_SNR_by_area_all.pdf'), bbox_inches='tight', dpi=300, transparent=True)

In [ ]:
# Fig Supp 2H
filt_monkeys_df_properties = all_monkey_df_properties[all_monkey_df_properties['control_shuffle']==False].copy()
x_order = ['L','A','D']
xticklabels = {'L':'L','A':'A','D':'D'}
hue_order = ['V4','V1']
hue = 'Area'	
y = '1-vs-rest r²'
neuron_property_label = y
x='Monkey Name'
fontsize=6
impose_y_lim = False
palette = ("#72BEB7", "#EDAEAE")
fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.violinplot(data=filt_monkeys_df_properties, x=x, y=y, hue=hue, ax=ax, alpha=0.7,
               inner_kws={'box_width':1.5, 'whis_width':0.7,
                                    'marker':'_', 'markersize':3,
                                    'markeredgewidth':0.5,
                                    },linewidth=0.5,cut=0,
               palette=palette, saturation=1, fill=False, log_scale=False)
sns.despine()
ax.tick_params(axis='both', labelsize=fontsize, width=0.5, length=2, pad=1.5)

ax.spines[:].set_linewidth(0.3)

ax.set(xlabel=None)
ax.set_ylabel(neuron_property_label, fontsize=fontsize, labelpad=0)
ax.set_xticks(ticks=ax.get_xticks(), labels=[xticklabels.get(lbl, lbl) for lbl in x_order])
if impose_y_lim is True:
	# Get the y-axis ticks
	y_ticks = plt.gca().get_yticks()
	# Check if 1 is among the ticks
	if 1 in y_ticks:
		ax.set_ylim(top=1)

ax.legend(title=None, fontsize=fontsize*0.8, labelspacing=0.2, handletextpad=0.3, borderpad=0.3)
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in x_order:
	pvals[x_i] = get_comparison_test_stars(filt_monkeys_df_properties[filt_monkeys_df_properties[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
											mouse_or_date='Date', perm_type='ind')
for i, act in enumerate(x_order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = filt_monkeys_df_properties[filt_monkeys_df_properties[x]==act][y].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs)
	if stars =="n.s.":
		y_top += 0.02*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)
	ax.legend_.get_frame().set_linewidth(0.3)
if save_figs:
	plt.savefig(os.path.join(fig_dir, 'figS3_monkey_1vsrest_all.pdf'), bbox_inches='tight', dpi=300, transparent=True)

In [ ]:
# Fig Supp 2I
filt_monkeys_df_properties = all_monkey_df_properties[all_monkey_df_properties['control_shuffle']==False].copy()
x_order = ['L','A','D']
xticklabels = {'L':'L','A':'A','D':'D'}
hue_order = ['V4','V1']
hue = 'Area'	
y = 'Variance w/in timepoint\nacross trials'
neuron_property_label = r'$\text{var}_{\text{repeats}}$'
x='Monkey Name'
fontsize=6
impose_y_lim = False
palette = ("#72BEB7", "#EDAEAE")
fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.violinplot(data=filt_monkeys_df_properties, x=x, y=y, hue=hue, ax=ax, alpha=0.7,
               inner_kws={'box_width':1.5, 'whis_width':0.7,
                                    'marker':'_', 'markersize':3,
                                    'markeredgewidth':0.5,
                                    },linewidth=0.5,cut=0,
               palette=palette, saturation=1, fill=False, log_scale=True)
sns.despine()
ax.tick_params(axis='both', labelsize=fontsize, width=0.5, length=2, pad=1.5)

ax.spines[:].set_linewidth(0.3)

ax.set(xlabel=None)
ax.set_ylabel(neuron_property_label, fontsize=fontsize, labelpad=0)
ax.set_xticks(ticks=ax.get_xticks(), labels=[xticklabels.get(lbl, lbl) for lbl in x_order])
if impose_y_lim is True:
	# Get the y-axis ticks
	y_ticks = plt.gca().get_yticks()
	# Check if 1 is among the ticks
	if 1 in y_ticks:
		ax.set_ylim(top=1)

ax.legend(title=None, fontsize=fontsize*0.8, labelspacing=0.2, handletextpad=0.3, borderpad=0.3)
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in x_order:
	pvals[x_i] = get_comparison_test_stars(filt_monkeys_df_properties[filt_monkeys_df_properties[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
											mouse_or_date='Date', perm_type='ind')
for i, act in enumerate(x_order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = filt_monkeys_df_properties[filt_monkeys_df_properties[x]==act][y].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs)
	if stars =="n.s.":
		y_top += 0.02*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)
	ax.legend_.get_frame().set_linewidth(0.3)
if save_figs:
	plt.savefig(os.path.join(fig_dir, 'figS3_monkey_var_across_repeats.pdf'), bbox_inches='tight', dpi=300, transparent=True)

In [ ]:
# Fig Supp 2J

filt_monkeys_df_properties = all_monkey_df_properties[all_monkey_df_properties['control_shuffle']==False].copy()
x_order = ['L','A','D']
xticklabels = {'L':'L','A':'A','D':'D'}
hue_order = ['V4','V1']
hue = 'Area'	
y = 'Variance w/in trial\nacross timepoints'
neuron_property_label = r'$\text{var}_{\text{time}}$'
x='Monkey Name'
fontsize=6
impose_y_lim = False
palette = ("#72BEB7", "#EDAEAE")
fig, ax = plt.subplots(figsize=(1.3,1.3))
sns.violinplot(data=filt_monkeys_df_properties, x=x, y=y, hue=hue, ax=ax, alpha=0.7,
               inner_kws={'box_width':1.5, 'whis_width':0.7,
                                    'marker':'_', 'markersize':3,
                                    'markeredgewidth':0.5,
                                    },linewidth=0.5,cut=0,
               palette=palette, saturation=1, fill=False, log_scale=True)
sns.despine()
ax.tick_params(axis='both', labelsize=fontsize, width=0.5, length=2, pad=1.5)

ax.spines[:].set_linewidth(0.3)

ax.set(xlabel=None)
ax.set_ylabel(neuron_property_label, fontsize=fontsize, labelpad=-2)
ax.set_xticks(ticks=ax.get_xticks(), labels=[xticklabels.get(lbl, lbl) for lbl in x_order])
if impose_y_lim is True:
	# Get the y-axis ticks
	y_ticks = plt.gca().get_yticks()
	# Check if 1 is among the ticks
	if 1 in y_ticks:
		ax.set_ylim(top=1)

ax.legend(title=None, fontsize=fontsize*0.8, labelspacing=0.2, handletextpad=0.3, borderpad=0.3)
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in x_order:
	pvals[x_i] = get_comparison_test_stars(filt_monkeys_df_properties[filt_monkeys_df_properties[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
											mouse_or_date='Date', perm_type='ind')
for i, act in enumerate(x_order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = filt_monkeys_df_properties[filt_monkeys_df_properties[x]==act][y].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs)
	if stars =="n.s.":
		y_top += 0.02*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)
	ax.legend_.get_frame().set_linewidth(0.3)
if save_figs:
	plt.savefig(os.path.join(fig_dir, 'figS3_monkey_var_across_time.pdf'), bbox_inches='tight', dpi=300, transparent=True)

In [ ]:
dir_monkey_df_gray_screen = make_monkey_df_directionality(all_monkey_stats[monkey_name], dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])
dir_monkey_df_lights_off = make_monkey_df_directionality(monkey_stats,  dataset_types=['RS'])
df_monkey_all_activity_types = pd.concat([dir_monkey_df_gray_screen,dir_monkey_df_lights_off], ignore_index=True)

In [ ]:
#Fig Supp 3B
fig, ax, pvals = plot_activitytype_by_direction_densities(
    animal_stats=df_monkey_all_activity_types,
    neuron_property="EV",
    x="Activity Type",
    hue="Direction",
    animal="monkey",               # or 'mouse'
    order=df_monkey_all_activity_types['Activity Type'].unique(),
    hue_order=["V1→V4","V4→V1"],  # optional
    draws_per_group=8000,
    bw="scott",
    mode="auto",
    num_permutations=10000,
    fig_size=(1.3, 1.3),
)
if save_figs is True:
	plt.savefig(fig_dir +'fig3_supp_monkey_comparisons_evar_all_activities.pdf',transparent=True,bbox_inches='tight' )

In [ ]:
## residuals Fig Supp 3F


monkey_name = 'L'
monkey_stats_path = os.path.join(project_root, 'results/fig_3',f'monkey_{monkey_name}_stats.pkl')
with open(monkey_stats_path, 'rb') as f:
    directionality_monkey_stats = pickle.load(f)
print("Loaded monkey stats from", monkey_stats_path)
dir_monkey_df_all = make_monkey_df_directionality(directionality_monkey_stats, dataset_types=['SNR','RF_thin','RF_large','SNR_spont','RS'], verbose=False)
# 1) Control for many numeric covariates
covs = ['SNR', 'Split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials']

dfr = residualize_within_condition_multi(
    dir_monkey_df_all, y='EV', xs=covs, cond='Dataset Type', flag_col='control_shuffle', flag_keep=False
)

fig, ax, pvals = plot_activitytype_by_direction_densities(
	animal_stats=dfr,
	neuron_property="EV_resid",
	x="Activity Type",
	hue="Direction",
	animal="monkey",               # or 'mouse'
	order=dfr['Activity Type'].unique(),
	hue_order=["V1→V4","V4→V1"],  # optional
	draws_per_group=8000,
	bw="scott",
	mode="auto",
	num_permutations=10000,
	fig_size=(1.2, 1.3),
	neuron_property_label="EV residual",
)
ax.set_xticklabels(['stimulus','gray\nscreen','lights\noff'], fontsize=6)
# ax.set_yticks([0,0.2,0.4,0.6,0.8,1])
# ax.set_yticklabels([])
# ax.set_ylabel('')
ax.set_ylabel('EV residual', fontsize=6, labelpad=-2)
ax.legend_.remove()
if save_figs is True:
	plt.savefig(fig_dir + f'fig3_supp_monkey_{monkey_name}_comparisons_evar_residuals_stim_GSLO.pdf',transparent=True,bbox_inches='tight' )

## Macaque A Directionality

### plotting

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]
dir_monkey_df_stimulus = make_monkey_df_directionality(monkey_stats)
dir_monkey_df_gray_screen = make_monkey_df_directionality(monkey_stats, dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])


#### supplemental

In [ ]:
#Fig Supp 3C
stratify_by_dict = {
	'stimulus': 'Date',
	'gray screen': 'Date',
}
dir_monkey_stim_gray = pd.concat([dir_monkey_df_stimulus,dir_monkey_df_gray_screen], ignore_index=True)
fig, ax, pvals = plot_activitytype_by_direction_densities(
	animal_stats=dir_monkey_stim_gray,
	neuron_property="EV",
	x="Activity Type",
	hue="Direction",
	animal="monkey",               # or 'mouse'
	order=dir_monkey_stim_gray['Activity Type'].unique(),
	hue_order=["V1→V4","V4→V1"],  # optional
	draws_per_group=8000,
	bw="scott",
	mode="auto",
	num_permutations=10000,
	fig_size=(0.9, 1.3),
	stratify=True,
	stratify_by_dict=stratify_by_dict,
)
ax.set_xticklabels(['stimulus','gray\nscreen'], fontsize=6)
if save_figs is True:
	plt.savefig(fig_dir + f'fig3_supp_monkey_{monkey_name}_comparisons_evar_stim_GS.pdf',transparent=True,bbox_inches='tight' )

In [ ]:
# residuals Fig Supp 3G

monkey_name = 'A'
monkey_stats_path = os.path.join(project_root, 'results/fig_3',f'monkey_{monkey_name}_stats.pkl')
with open(monkey_stats_path, 'rb') as f:
    directionality_monkey_stats = pickle.load(f)
print("Loaded monkey stats from", monkey_stats_path)
dir_monkey_df_all = make_monkey_df_directionality(directionality_monkey_stats, dataset_types=['SNR','RF_thin','RF_large','SNR_spont','RF_thin_spont','RF_large_spont'], verbose=False)
# 1) Control for many numeric covariates
covs = ['SNR', 'Split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials']

dfr = residualize_within_condition_multi(
    dir_monkey_df_all, y='EV', xs=covs, cond='Dataset Type', flag_col='control_shuffle', flag_keep=False
)

fig, ax, pvals = plot_activitytype_by_direction_densities(
	animal_stats=dfr,
	neuron_property="EV_resid",
	x="Activity Type",
	hue="Direction",
	animal="monkey",               # or 'mouse'
	order=dfr['Activity Type'].unique(),
	hue_order=["V1→V4","V4→V1"],  # optional
	draws_per_group=8000,
	bw="scott",
	mode="auto",
	num_permutations=10000,
	fig_size=(0.9, 1.3),
	neuron_property_label="EV residual",
)
ax.set_xticklabels(['stimulus','gray\nscreen','lights\noff'], fontsize=6)
ax.legend_.remove()
ax.set_ylabel('EV residual', fontsize=6, labelpad=-2)

if save_figs is True:
	plt.savefig(fig_dir + f'fig3_supp_monkey_{monkey_name}_comparisons_evar_residuals_stim_GSLO.pdf',transparent=True,bbox_inches='tight' )

## Macaque $L_{A}$ Directionality

### plotting

In [ ]:
with open(os.path.join(project_root, 'results/fig_3', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'A'

# with stratify 
stratify_by_dict = {
	'stimulus': 'Date',
	'gray screen': 'Date',
}

dir_stimulus_df_monkey_L_A = pd.DataFrame()
stim_gray_pvals = {'stimulus':[], 'gray screen':[]}

for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_3/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	
	dir_monkey_df_stim_gray_subsampled_monkey_df = make_monkey_df_directionality(subsampled_monkey_stats, dataset_types=all_datataset_types_)
	dir_monkey_df_stim_gray_subsampled_monkey_df['Subsample Seed'] = seed
	dir_stimulus_df_monkey_L_A = pd.concat([dir_stimulus_df_monkey_L_A, dir_monkey_df_stim_gray_subsampled_monkey_df], ignore_index=True)
	fig, ax, pvals = plot_activitytype_by_direction_densities(
		animal_stats=dir_monkey_df_stim_gray_subsampled_monkey_df,
		neuron_property="EV",
		x="Activity Type",
		hue="Direction",
		animal="monkey",               # or 'mouse'
		order=dir_monkey_df_stim_gray_subsampled_monkey_df['Activity Type'].unique(),
		hue_order=["V1→V4","V4→V1"],  # optional
		draws_per_group=8000,
		bw="scott",
		mode="auto",
		num_permutations=10000,
		fig_size=(0.9, 1.3),
		stratify=True,
		stratify_by_dict=stratify_by_dict,
		subsampled_indices=True,
		omit_figs=True,
)
	stim_gray_pvals['stimulus'].append(pvals['stimulus'])
	stim_gray_pvals['gray screen'].append(pvals['gray screen'])

In [ ]:
#Fig Supp 3D
stratify_by_dict = {
	'stimulus': 'Subsample Seed',
	'gray screen': 'Subsample Seed',
}
fig, ax, _ = plot_activitytype_by_direction_densities(
		animal_stats=dir_stimulus_df_monkey_L_A,
		neuron_property="EV",
		x="Activity Type",
		hue="Direction",
		animal="monkey",               # or 'mouse'
		order=dir_stimulus_df_monkey_L_A['Activity Type'].unique(),
		hue_order=["V1→V4","V4→V1"],  # optional
		draws_per_group=8000,
		bw="scott",
		mode="auto",
		num_permutations=10000,
		fig_size=(0.9, 1.3),
		stratify=True,
		stratify_by_dict=stratify_by_dict,
		subsampled_indices=True,
		show_pvals=False
)

pvals = {'stimulus': np.median(stim_gray_pvals['stimulus']), 'gray screen': np.median(stim_gray_pvals['gray screen'])}

def p_to_stars(p):
	return "***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "n.s."))
# compute y for stars a bit above the taller of the pair
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins

for i, act in enumerate(dir_stimulus_df_monkey_L_A['Activity Type'].unique()):
	if act not in pvals:
		continue
	p = pvals[act]
	stars = p_to_stars(p)
	# find max y at this category from plotted data to place stars above
	sub = dir_stimulus_df_monkey_L_A[dir_stimulus_df_monkey_L_A['Activity Type']==act]['EV'].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=6, color=color)
ax.set_xticklabels(['stimulus','gray\nscreen'], fontsize=6)
ax.legend_.remove()
if save_figs is True:
	plt.savefig(fig_dir + f'fig3_supp_monkey_L_subsampled_A_comparisons_evar_stim_GS.pdf',transparent=True,bbox_inches='tight' )


In [ ]:

with open(os.path.join(project_root, 'results/fig_3', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)

main_monkey_name = 'L'
subsample_monkey_name = 'A'
dir_stimulus_df_monkey_L_A_residuals= pd.DataFrame()
stim_gray_pvals = {'stimulus':[], 'gray screen':[]}

for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_3/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]

	dir_monkey_df_all = make_monkey_df_directionality(subsampled_monkey_stats, dataset_types=all_datataset_types_, verbose=False)
	# 1) Control for many numeric covariates
	covs = ['SNR', 'Split-half r',
		'Variance w/in trial\nacross timepoints',
		'Variance w/in timepoint\nacross trials']

	dfr = residualize_within_condition_multi(
		dir_monkey_df_all, y='EV', xs=covs, cond='Dataset Type', flag_col='control_shuffle', flag_keep=False
	)
	dfr['Subsample Seed'] = seed
	fig, ax, pvals = plot_activitytype_by_direction_densities(
		animal_stats=dfr,
		neuron_property="EV_resid",
		x="Activity Type",
		hue="Direction",
		animal="monkey",               # or 'mouse'
		order=dfr['Activity Type'].unique(),
		hue_order=["V1→V4","V4→V1"],  # optional
		draws_per_group=8000,
		bw="scott",
		mode="auto",
		num_permutations=10000,
		fig_size=(0.9, 1.3),
		neuron_property_label="EV residual",
		subsampled_indices=True,
		omit_figs=True
	)
	stim_gray_pvals['stimulus'].append(pvals['stimulus'])
	stim_gray_pvals['gray screen'].append(pvals['gray screen'])
	dir_stimulus_df_monkey_L_A_residuals = pd.concat([dir_stimulus_df_monkey_L_A_residuals, dfr], ignore_index=True)

In [ ]:
#Fig Supp 3H
fig, ax, pvals = plot_activitytype_by_direction_densities(
	animal_stats=dir_stimulus_df_monkey_L_A_residuals,
	neuron_property="EV_resid",
	x="Activity Type",
	hue="Direction",
	animal="monkey",               # or 'mouse'
	order=dir_stimulus_df_monkey_L_A_residuals['Activity Type'].unique(),
	hue_order=["V1→V4","V4→V1"],  # optional
	draws_per_group=8000,
	bw="scott",
	mode="auto",
	num_permutations=10000,
	fig_size=(0.9, 1.3),
	neuron_property_label="EV residual",
	stratify=True,
	stratify_by_dict={'stimulus':'Subsample Seed','gray screen':'Subsample Seed','lights off':'Subsample Seed'},
	show_pvals=False
)

pvals = {'stimulus': np.median(stim_gray_pvals['stimulus']), 'gray screen': np.median(stim_gray_pvals['gray screen'])}

ax.set_xticklabels(['stimulus','gray\nscreen'], fontsize=6)
ax.legend_.remove()
ax.set_ylabel('EV residual', fontsize=6, labelpad=-4)

def p_to_stars(p):
	return "***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "n.s."))
# compute y for stars a bit above the taller of the pair
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins

for i, act in enumerate(dir_stimulus_df_monkey_L_A_residuals['Activity Type'].unique()):
	if act not in pvals:
		continue
	p = pvals[act]
	stars = p_to_stars(p)
	# find max y at this category from plotted data to place stars above
	sub = dir_stimulus_df_monkey_L_A_residuals[dir_stimulus_df_monkey_L_A_residuals['Activity Type']==act]['EV_resid'].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=6, color=color)
ax.set_xticklabels(['stimulus','gray\nscreen'], fontsize=6)
if save_figs is True:
	plt.savefig(fig_dir + f'fig3_supp_monkey_L_subsampled_A_comparisons_evar_residuals_stim_GSLO.pdf',transparent=True,bbox_inches='tight' )